# Building a Recommender System on the Movie Lens dataset 1 - Cleaning the movies table

**Subject** - Hired by a Parisian cinema, you want to recommend VOD movies to the clients based on what they went to see during the week. What do you recommend for someone who just saw Inferno, by Ron Howard ?

**Data** - First start by downloading the MovieLens 20M Dataset (https://grouplens.org/datasets/movielens/) and place the files into your project directory, in a subdirectory named 'data'

**This notebook focuses on treating and cleaning the "Movies" table**

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

In [2]:
project_dir = '/Users/nicolasjudalet/Desktop/Ignition Challenge/'
df_movies = pd.read_csv(project_dir+'data/movies.csv', encoding = 'utf-8', delimiter=',')

In [3]:
df_movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
df_movies.isnull().sum() # there are no missing values in the movies table

movieId    0
title      0
genres     0
dtype: int64

It looks like the title column and genres columns can be parsed to obtain information resp. on the year of the movie and the different genres it belongs to.

### Extract year from title of the movies

In [5]:
import re
titles = np.zeros(len(df_movies.title))
no_match_count = 0
for i, title in enumerate(df_movies.title):
    pat = re.compile('\([0-9]{4}\)\Z')
    year_found = pat.search(string=title)
    if year_found:
        titles[i] = re.sub(pattern='[()]', repl='', string=year_found.group())*1
    else:
        no_match_count += 1
print('No match count : ', no_match_count)

No match count :  229


It seems that the year was found in most titles, except 229 of them. Let us have a look at them

In [6]:
df_no_match = df_movies[titles == 0]
#print(df_no_match.head())
#print(df_no_match.tail())

Indeed, some titles seem to have no date : it is understandable that the search did not match.
Others seem to have a date in the right format... Let us try to understand why the match wes not performed.

In [7]:
test = df_no_match.title[8495]
test

'Babe Ruth Story, The (1948) '

The match failed because of a space character at the end of the name. Let'us try to add this case into our search.

In [8]:
no_match_count = 0
for title in df_no_match.title:
    index = df_no_match[df_no_match.title == title].index.values
    pat = re.compile('\([0-9]{4}\) *')
    year_found = pat.search(string=title)
    if year_found:
        titles[index] = re.sub(pattern='[() ]', repl='', string=year_found.group())*1
    else:
        no_match_count += 1
print('No match count : ', no_match_count)

No match count :  22


There are now only 22 rows with no year found. Let us do a final check on them. 

In [9]:
df_no_match = df_movies[titles == 0]
#df_no_match

Most of them do not actually have a year indicated in the title. Three of them can be added manually to the year column, with the mean year of all years between the start year indicated and the last year of the dataset, i.e. 2015

In [10]:
titles[22368] = 2012 # Diplomatic Immunity
titles[22669] = 2011 # Big Bang Theory
titles[22679] = 1977 # Fawlty Towers

In [11]:
df_movies['year'] = titles
df_movies['year'] = df_movies.year.astype(int)
df_movies.loc[df_movies.year == 0, 'year'] = np.nan
#df_movies

### Separate Genres into different columns

In [21]:
#df_movies.genres.value_counts()

It seems that the genres are separated by the character '|' and not so many of them are present in the dataset. From the first data inspection, we also know that some movies present the value '(no genres listed)'. We are first going to extract the whole vocabulary that is used to describe genres.

In [13]:
genres_voc = set('|'.join(df_movies.genres.tolist()).split('|'))
genres_voc.remove('(no genres listed)')
# genres_voc

In [14]:
def find_word(row, word):
    return(word in row.genres)

for genre in genres_voc:
    df_movies['genre_' + genre] = df_movies.apply(lambda row: find_word(row, word=genre), axis = 1)

df_movies = df_movies.drop('genres', axis = 1)
#df_movies.head()

In [17]:
np.max(df_movies['year'])

2015.0

### Write down results in csv file

In [19]:
df_movies.to_csv(project_dir+'data/movies_cleaned.csv', encoding = 'utf-8', sep=',', index=False)